In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data_path = 'C:/Users/mccal/Desktop/Springboard Files/Springboard 26 SPARK SQL/Ex_Files_Spark_SQL_DataFrames/Exercise Files/Data'

json_df2_path = data_path+'/utilization.json'
df_util = spark.read.format('json').load(json_df2_path)

df_util.show(10)

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
+---------------+-------------------+-

In [2]:
df_util.createOrReplaceTempView("utilization")

In [4]:
spark.sql("SELECT server_id, min(cpu_utilization), max(cpu_utilization),  stddev(cpu_utilization) \
           From utilization \
           GROUP BY server_id").show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      112|                0.52|                0.92|    0.11528867845082576|
|      113|                0.58|                0.98|    0.11544345150353687|
|      130|                0.35|                0.75|    0.11568834774245991|
|      126|                0.48|                0.88|    0.11542612970702058|
|      149|                0.54|                0.94|    0.11543517500295467|
|      110|                0.35|                0.75|    0.11533251724450215|
|      136|                0.41|                 0.8|    0.11597405743182258|
|      144|                0.47|                0.87|    0.11478654960489501|
|      119|                0.22|                0.62|    0.11516031929842008|
|      116|                 0.3|                 0.7|    0.11506

In [6]:
avg_window = spark.sql("SELECT event_datetime, server_id, cpu_utilization, \
           avg(cpu_utilization)  OVER (PARTITION BY server_id) avg_server_util \
           FROM utilization")

In [7]:
avg_window.show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:34|      112|           0.71|0.7153870000000067|
|03/05/2019 08:11:34|      112|           0.78|0.7153870000000067|
|03/05/2019 08:16:34|      112|           0.87|0.7153870000000067|
|03/05/2019 08:21:34|      112|           0.82|0.7153870000000067|
|03/05/2019 08:26:34|      112|           0.62|0.7153870000000067|
|03/05/2019 08:31:34|      112|            0.9|0.7153870000000067|
|03/05/2019 08:36:34|      112|           0.89|0.7153870000000067|
|03/05/2019 08:41:34|      112|           0.81|0.7153870000000067|
|03/05/2019 08:46:34|      112|           0.88|0.7153870000000067|
|03/05/2019 08:51:34|      112|           0.89|0.7153870000000067|
|03/05/2019 08:56:34|      112|           0.84|0.7153870000000067|
|03/05/2019 09:01:34|      112|           0.71|0.7153870000000

In [9]:
avg_window2 = spark.sql("SELECT event_datetime, server_id, cpu_utilization, \
           avg(cpu_utilization)  OVER (PARTITION BY server_id) avg_server_util, \
           cpu_utilization - avg(cpu_utilization)  OVER (PARTITION BY server_id) delta_server_util \
           FROM utilization")

In [10]:
avg_window2.show()

+-------------------+---------+---------------+------------------+--------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|   delta_server_util|
+-------------------+---------+---------------+------------------+--------------------+
|03/05/2019 08:06:34|      112|           0.71|0.7153870000000067|-0.00538700000000...|
|03/05/2019 08:11:34|      112|           0.78|0.7153870000000067| 0.06461299999999337|
|03/05/2019 08:16:34|      112|           0.87|0.7153870000000067| 0.15461299999999334|
|03/05/2019 08:21:34|      112|           0.82|0.7153870000000067|  0.1046129999999933|
|03/05/2019 08:26:34|      112|           0.62|0.7153870000000067|-0.09538700000000666|
|03/05/2019 08:31:34|      112|            0.9|0.7153870000000067| 0.18461299999999337|
|03/05/2019 08:36:34|      112|           0.89|0.7153870000000067| 0.17461299999999336|
|03/05/2019 08:41:34|      112|           0.81|0.7153870000000067|  0.0946129999999934|
|03/05/2019 08:46:34|      112| 

In [19]:
sql_window3 =  spark.sql("SELECT event_datetime, server_id, cpu_utilization,  \
                      avg(cpu_utilization) OVER (PARTITION BY server_id ORDER BY event_datetime \
                                    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) avg_server_util \
                      FROM utilization")

In [20]:
sql_window3.show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:34|      112|           0.71|             0.745|
|03/05/2019 08:11:34|      112|           0.78|0.7866666666666666|
|03/05/2019 08:16:34|      112|           0.87|0.8233333333333333|
|03/05/2019 08:21:34|      112|           0.82|              0.77|
|03/05/2019 08:26:34|      112|           0.62|0.7799999999999999|
|03/05/2019 08:31:34|      112|            0.9|0.8033333333333333|
|03/05/2019 08:36:34|      112|           0.89|0.8666666666666667|
|03/05/2019 08:41:34|      112|           0.81|              0.86|
|03/05/2019 08:46:34|      112|           0.88|              0.86|
|03/05/2019 08:51:34|      112|           0.89|              0.87|
|03/05/2019 08:56:34|      112|           0.84|0.8133333333333334|
|03/05/2019 09:01:34|      112|           0.71|0.7999999999999